In [8]:
import ee
import geemap
import requests
import os

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='btech-project-443813')  # Make sure this project ID is correct

# Step 2: Define the full region of interest (Delhi)
full_roi = ee.Geometry.Rectangle([76.8, 28.4, 77.4, 28.9])  # Entire area

# Step 3: Divide the region into smaller tiles (2x2 grid)
tiles = [
    ee.Geometry.Rectangle([76.8, 28.4, 77.1, 28.65]),  # Bottom-left
    ee.Geometry.Rectangle([77.1, 28.4, 77.4, 28.65]),  # Bottom-right
    ee.Geometry.Rectangle([76.8, 28.65, 77.1, 28.9]),  # Top-left
    ee.Geometry.Rectangle([77.1, 28.65, 77.4, 28.9])   # Top-right
]

# Step 4: Download each tile and save locally
output_folder = r"/home/stormej/dev/varsha/data/dem/dem_tif"  # Update this path
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

def download_tile(geometry, filename):
    """Download a tile as GeoTIFF."""
    srtm = ee.Image('USGS/SRTMGL1_003').clip(geometry)
    url = srtm.getDownloadURL({
        'scale': 90,
        'region': geometry.getInfo()['coordinates'],
        'format': 'GEO_TIFF'
    })

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {filename}")

# Loop over tiles and download them
for i, tile in enumerate(tiles):
    filename = os.path.join(output_folder, f"srtm_tile_{i + 1}.tif")
    download_tile(tile, filename)

print("All tiles downloaded successfully!")


Downloaded: /home/stormej/dev/varsha/data/dem/dem_tif/srtm_tile_1.tif
Downloaded: /home/stormej/dev/varsha/data/dem/dem_tif/srtm_tile_2.tif
Downloaded: /home/stormej/dev/varsha/data/dem/dem_tif/srtm_tile_3.tif
Downloaded: /home/stormej/dev/varsha/data/dem/dem_tif/srtm_tile_4.tif
All tiles downloaded successfully!


In [10]:
import rasterio
from rasterio.merge import merge
import glob

# List resampled files
tile_files = glob.glob(r"/home/stormej/dev/varsha/data/dem/dem_tif/*.tif")

# Open files and prepare for merging
src_files_to_mosaic = [rasterio.open(fp) for fp in tile_files]
mosaic, out_transform = merge(src_files_to_mosaic)

# Save the merged mosaic
output_merged_path = r"/home/stormej/dev/varsha/data/dem/merged_dem.tif"

# Use the correct dimensions and metadata
with rasterio.open(
    output_merged_path,
    'w',
    driver='GTiff',
    height=mosaic.shape[1],  # height = 8660
    width=mosaic.shape[2],   # width = 5567
    count=mosaic.shape[0],   # count = 1 (number of bands)
    dtype=mosaic.dtype,
    crs=src_files_to_mosaic[0].crs,
    transform=out_transform,
) as dest:
    dest.write(mosaic)

print(f"Merged DEM saved at: {output_merged_path}")


         


Merged DEM saved at: /home/stormej/dev/varsha/data/dem/merged_dem.tif


In [11]:
import os
import rasterio
from rasterio.enums import Resampling

# Paths
rainfall_folder = r'/home/stormej/dev/varsha/data/reference_tif'  # Folder containing rainfall files
dem_path = r"/home/stormej/dev/varsha/data/dem/merged_dem.tif"  # Replace with your DEM file path
output_folder = r'/home/stormej/dev/varsha/data/dem'  # Folder to save resampled DEM

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process the DEM file
print(f"\nProcessing DEM file")

with rasterio.open(dem_path) as dem_src:
    dem_data = dem_src.read(1)
    print(f"DEM data shape: {dem_data.shape}, dtype: {dem_data.dtype}")
    
    # Resample DEM data using bilinear interpolation
    # Using bilinear instead of average as it's more appropriate for elevation data
    resampled_dem = dem_src.read(
        out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
        resampling=Resampling.bilinear
    )
    print(f"Resampled DEM shape: {resampled_dem.shape}, dtype: {resampled_dem.dtype}")
    
    # Create output filename
    output_filename = 'dem_resampled.tif'
    output_path = os.path.join(output_folder, output_filename)
    
    # Save the resampled DEM
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=resampled_dem.shape[1],
        width=resampled_dem.shape[2],
        count=1,
        dtype=resampled_dem.dtype,
        crs=rainfall_crs,
        transform=rainfall_transform,
        compress='lzw'  # Added compression for smaller file size
    ) as dst:
        dst.write(resampled_dem[0], 1)

print(f"Resampled DEM saved to: {output_path}")
print("\nDEM resampling completed.")

Using reference rainfall file: /home/stormej/dev/varsha/data/reference_tif/refrence_tif.tif
Reference rainfall data shape: (129, 135), CRS: EPSG:4326

Processing DEM file
DEM data shape: (619, 743), dtype: int16
Resampled DEM shape: (1, 129, 135), dtype: int16
Resampled DEM saved to: /home/stormej/dev/varsha/data/dem/dem_resampled.tif

DEM resampling completed.
